# Simple GP Regression 

Gaussian processes are a powerful tool for nonlinear regression models. 

Assume that we have **predictor** variables $\mathbf{X} = \{\mathbf{x_i}\}_{i=1}^N \in \mathbb{R}^d$ and **response** variables $\mathbf{y}=\{y_i \in \mathbb{R}\}_{i=1}^N$.

The response variables $\mathbf{y}$ are assumed to dependent on the predictors $\mathbf{X}$,

\begin{equation}
y_i \sim \mathcal{N}(f(\mathbf{x}_i),\sigma^2), \ i=1,\ldots,n,  
\end{equation}

where $f$ is a mapping function. Treating $f$ as a random function, we assume that the distribution over $f$ is a **Gaussian process**,
$$
f \sim \mathcal{GP}(m(\mathbf{x}),k(\mathbf{x},\mathbf{x}')),
$$
where $m(\cdot)$ and $k(\cdot,\cdot)$ are the mean and kernel functions respectively.

## 1D regression example

We start by simulating some data

In [13]:
using PyPlot, GaussianProcesses

srand(13579)
# Training data
n=10;                          #number of training points
x = 2π * rand(n);              #predictors
y = sin(x) + 0.05*randn(n);    #regressors

The first step in modelling with Gaussian Processes is to choose mean functions and kernels which describe the process. GaussianProcesses can be optionally used with a plotting package. Currently the packages [Gadfly](https://github.com/dcjones/Gadfly.jl) and [PyPlot](https://github.com/stevengj/PyPlot.jl) are supported.

**Note** that all hyperparameters for the **mean** and **kernel** functions and $\sigma$ are given on the log scale. This is true for all strictly positive hyperparameters. Gaussian Processes are represented by objects of type 'GP' and constructed from observation data, a mean function and kernel, and optionally the amount of observation noise.

In [14]:
#Select mean and covariance function
mZero = MeanZero()                   #Zero mean function
kern = SE(0.0,0.0)                   #Sqaured exponential kernel (note that hyperparameters are on the log scale)

logObsNoise = -1.0                        # log standard deviation of observation noise (this is optional)
gp = GP(x,y,mZero,kern,logObsNoise)       #Fit the GP

GP Exact object:
  Dim = 1
  Number of observations = 10
  Mean function:
    Type: GaussianProcesses.MeanZero, Params: Float64[]
  Kernel:
    Type: GaussianProcesses.SEIso, Params: [0.0,0.0]
  Input observations = 
[5.66072 1.67222 … 6.08978 3.39451]
  Output observations = [-0.505287,1.02312,0.616955,-0.777658,-0.875402,0.92976,-1.04525,-0.00543704,-0.301759,-0.364661]
  Variance of observation noise = 0.1353352832366127
  Marginal Log-Likelihood = -6.719

Once we've fit the `GP` function to the data, we can calculate the **predicted mean** and **variance** of of the function at unobserved points $\{\mathbf{x}^*,y^*\}$, conditional on the observed data $\mathcal{D}=\{\mathbf{y},\mathbf{X}\}$. This is done with the `predict_y` function.

The `predict_y` function returns the mean vector $\mu(\mathbf{x}^*)$ and covariance matrix (variance vector if `full_cov=false`) $\Sigma(\mathbf{x}^*,\mathbf{x}^{*^\top})$ of the predictive distribution,
\begin{equation}
    y^*|\mathbf{x^*},\mathcal{D} \sim \mathcal{N}(\mu(\mathbf{x}^*),\Sigma(\mathbf{x}^*,\mathbf{x}^{*^\top})+\sigma^2\mathbf{I}),
\end{equation}
where 
\begin{align}
\mu(\mathbf{x}^*) &= k(\mathbf{x}^*,\mathbf{X})(k(\mathbf{X}, \mathbf{X}) + \sigma_n^2 \mathbf{I})^{-1}\mathbf{y} \\ \Sigma(\mathbf{x}^*,\mathbf{x}^{*'}) &= k(\mathbf{x}^*,\mathbf{x}^*) -k(\mathbf{x}^*,\mathbf{X})(k(\mathbf{X}, \mathbf{X})+ \sigma_n^2 \mathbf{I})^{-1} k(\mathbf{X},\mathbf{x}^*).
\end{align}

**Note** you can use the `predict_f` function to predict the latent function $\mathbf{f}^*$.

In [15]:
μ, σ² = predict_y(gp,linspace(0,2π,100));

The predict function is implicitly used when plotting the GP. Plotting is straightforward to apply, but the display will depend on the package loaded at the start of the session (e.g. PyPlot or Gadfly). Note that, at present, the plotting package should be loaded before `GaussianProcesses`. The plot function outputs the predicted mean (blue line) and the uncertainty in the function is given by the confidence bands, which are set to 95% by default.

In [16]:
plot(gp)                          #Plot the GP

LoadError: PyError (:PyObject_Call) <type 'exceptions.TypeError'>
TypeError('float() argument must be a string or a number',)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py", line 3161, in plot
    ret = ax.plot(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py", line 1819, in inner
    return func(ax, *args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_axes.py", line 1383, in plot
    self.add_line(line)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_base.py", line 1703, in add_line
    self._update_line_limits(line)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_base.py", line 1725, in _update_line_limits
    path = line.get_path()
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/lines.py", line 938, in get_path
    self.recache()
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/lines.py", line 634, in recache
    y = np.asarray(yconv, np.float_)


The hyperparameters are optimized using the [Optim](https://github.com/JuliaOpt/Optim.jl) package. This offers users a range of optimization algorithms which can be applied to estimate the hyperparameters using type II maximum likelihood estimation. Gradients are available for all mean and kernel functions used in the package and therefore it is recommended that the user utilizes gradient based optimization techniques. As a default, the `optimize!` function uses the `Conjugate Gradients` solver, however, alternative solvers can be applied. 

In [17]:
optimize!(gp; method=Optim.BFGS())   #Optimise the hyperparameters

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [-1.0,0.0,0.0]
 * Minimizer: [-2.681987357828038,0.434215173836849, ...]
 * Minimum: -4.902989e-01
 * Iterations: 9
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 40
 * Gradient Calls: 40

In [18]:
plot(gp)   #Plot the GP after the hyperparameters have been optimised 

LoadError: PyError (:PyObject_Call) <type 'exceptions.TypeError'>
TypeError('float() argument must be a string or a number',)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py", line 3161, in plot
    ret = ax.plot(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py", line 1819, in inner
    return func(ax, *args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_axes.py", line 1383, in plot
    self.add_line(line)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_base.py", line 1703, in add_line
    self._update_line_limits(line)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_base.py", line 1725, in _update_line_limits
    path = line.get_path()
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/lines.py", line 938, in get_path
    self.recache()
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/lines.py", line 634, in recache
    y = np.asarray(yconv, np.float_)


## Multi-dimensional regression

The regression example above can be easily extended to higher dimensions. For the purpose of visualisation, and without loss of generality, we consider a 2 dimensional regression example.

In [19]:
#Training data
d, n = 2, 50;         #Dimension and number of observations
x = 2π * rand(d, n);                               #Predictors
y = vec(sin(x[1,:]).*sin(x[2,:])) + 0.05*rand(n);  #Responses

For problems of dimension>1 we can use isotropic (`Iso`) kernels or automatic relevance determination (`ARD`) kernels. For `Iso` kernels, the length scale parameter $\ell$ is the same for all dimensions. For `ARD` kernels, each dimension has different length scale parameter.

The `Iso` and `ARD` kernels are implemented automatically by replacing the single length scale parameter with a vector of parameters. For example, below we use the Matern 5/2 `ARD` kernel, if we wanted to use the `Iso` alternative then we would set the kernel as `kern=Mat(5/2,0.0,0.0)`.

In this example we use a composite kernel represented as the sum of a Matern 5/2 ARD kernel and a Squared Exponential isotropic kernel. This is easily implemented using the `+` symbol, or in the case of a product kernel, using the `*` symbol (i.e. `kern = Mat(5/2,[0.0,0.0],0.0) * SE(0.0,0.0)`).

In [20]:
mZero = MeanZero()                             # Zero mean function
kern = Mat(5/2,[0.0,0.0],0.0) + SE(0.0,0.0)    # Sum kernel with Matern 5/2 ARD kernel 
                                               # with parameters [log(ℓ₁), log(ℓ₂)] = [0,0] and log(σ) = 0
                                               # and Squared Exponential Iso kernel with
                                               # parameters log(ℓ) = 0 and log(σ) = 0

Type: GaussianProcesses.SumKernel
  Type: GaussianProcesses.Mat52Ard, Params: [-0.0,-0.0,0.0]
  Type: GaussianProcesses.SEIso, Params: [0.0,0.0]


Fit the Gaussian process to the data using the prespecfied mean and covariance functions.

In [21]:
gp = GP(x,y,mZero,kern,-2.0)          # Fit the GP

GP Exact object:
  Dim = 2
  Number of observations = 50
  Mean function:
    Type: GaussianProcesses.MeanZero, Params: Float64[]
  Kernel:
    Type: GaussianProcesses.SumKernel
      Type: GaussianProcesses.Mat52Ard, Params: [-0.0,-0.0,0.0]
      Type: GaussianProcesses.SEIso, Params: [0.0,0.0]
  Input observations = 
[1.43935 0.515919 … 0.487943 2.38029; 4.21509 3.59947 … 0.699132 4.09243]
  Output observations = [-0.850956,-0.173897,0.555086,-0.579203,-0.116111,-0.304235,-0.192372,-0.55408,0.649668,-0.226597  …  -0.874219,-0.136116,0.485536,0.571545,-0.387655,0.200481,-0.33697,0.395318,0.340145,-0.545092]
  Variance of observation noise = 0.01831563888873418
  Marginal Log-Likelihood = -24.334

Using the [Optim](https://github.com/JuliaOpt/Optim.jl) package we have the option to choose from a range of optimize functions including conjugate gradients. It is also possible to fix the hyperparameters in either the mean function, kernel function or observation noise, by settting them to false in `optimize!` (e.g. `optimize!(...,mean=false)`).

In [22]:
optimize!(gp)                         # Optimize the hyperparameters

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-2.0,-0.0,-0.0,0.0,0.0,0.0]
 * Minimizer: [-4.398843024597842,-0.5062449599019844, ...]
 * Minimum: -5.401611e+01
 * Iterations: 29
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 120
 * Gradient Calls: 120

In [23]:
plot(gp; clim=(-10.0, 10.0,-10.0,10.0)) #Plot the GP over range clim

LoadError: PyError (:PyObject_Call) <type 'exceptions.AttributeError'>
AttributeError(u'Unknown property clim',)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py", line 3161, in plot
    ret = ax.plot(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py", line 1819, in inner
    return func(ax, *args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_axes.py", line 1382, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_base.py", line 381, in _grab_next_args
    for seg in self._plot_args(remaining, kwargs):
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_base.py", line 369, in _plot_args
    seg = func(x[:, j % ncx], y[:, j % ncy], kw, kwargs)
  File "/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_base.py", line 276, in _makeline
    seg = mlines.Line2D(x, y, **kw)
  File "/usr/local/lib/python2.7/dist-packages/ma